Nama: Syarani Afa Natira Kusumah<br>
NIM: 2502037864<br>
Kelas: LB06

**Case No. 2 (BERT)**

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import re
import string
string.punctuation
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, AdamW
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# a) Perform Preprocess and prepare Dataset step, describe each step including some concepts in Natural Language Processing (NLP)

In [ ]:
# Load and preprocess data
df = pd.read_csv("Emotion.csv")
df.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
df.isna().sum()

text     0
label    0
dtype: int64

In [ ]:
df['label'].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [ ]:
# Encoding
category_to_numerical = {
    "anger":0,
    "fear":1,
    "joy":2,
    "love":3,
    "sadness":4,
    "surprise":5
}

df["label"] = df["label"].map(category_to_numerical)
df.tail()

,text,label
19995,im having ssa examination tomorrow in the morn...,4
19996,i constantly worry about their fight against n...,2
19997,i feel its important to share this info for th...,2
19998,i truly feel that if you are passionate enough...,2
19999,i feel like i just wanna buy any cute make up ...,2


In [ ]:
# Text cleaning
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

df['text'] = df['text'].apply(remove_punctuation)
df.head()

,text,label
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0


Disini saya melakukan preprocessing data, seperti melihat missing value, melihat label pada dataset, serta meremove special character pada teks. Selanjutnya, saya membagi dataset seperti yang tertera pada soal lalu membuat tokenisasi teks menggunakan BertTokenizer. BertTokenizer digunakan untuk memproses teks dan mengonversi teks menjadi representasi numerik yang dapat dimengerti oleh model BERT.

# b) Divide the dataset into training, validation, and testing sets following these guidelines: allocate 70% of the data for training, 15% for validation, and 15% for testing

In [ ]:
train_size = 0.7
test_size = 0.15
val_size= 0.15

train_data, test_data = train_test_split(df, train_size=train_size, random_state=42)
val_data, test_data = train_test_split(test_data, train_size=test_size/(test_size + val_size), random_state=42)

print(f"Number of data train: {len(train_data)}")
print(f"Number of data test: {len(test_data)}")
print(f"Number of data validation: {len(val_data)}")

Number of data train: 14000
Number of data test: 3000
Number of data validation: 3000


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_tokens = tokenizer(train_data['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
val_tokens = tokenizer(val_data['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
test_tokens = tokenizer(test_data['text'].tolist(), padding=True, truncation=True, return_tensors='tf')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
x_train = {
    'input_ids': train_tokens['input_ids'],
    'token_type_ids': train_tokens['token_type_ids'],
    'attention_mask': train_tokens['attention_mask']
}
y_train = tf.convert_to_tensor(train_data['label'].values)

x_val = {
    'input_ids': val_tokens['input_ids'],
    'token_type_ids': val_tokens['token_type_ids'],
    'attention_mask': val_tokens['attention_mask']
}
y_val = tf.convert_to_tensor(val_data['label'].values)

x_test = {
    'input_ids': test_tokens['input_ids'],
    'token_type_ids': test_tokens['token_type_ids'],
    'attention_mask': test_tokens['attention_mask']
}
y_test = tf.convert_to_tensor(test_data['label'].values)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# c) Develop the model based on Table 1. Train your model based on the train dataset until achieving satisfactory accuracy.

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
num_epochs = 5
batch_size = 32

train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [ ]:
history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/5
438/438 [==============================] - 170s 275ms/step - loss: 0.8736 - accuracy: 0.7039 - val_loss: 0.3139 - val_accuracy: 0.8977
Epoch 2/5
438/438 [==============================] - 110s 251ms/step - loss: 0.2213 - accuracy: 0.9244 - val_loss: 0.1912 - val_accuracy: 0.9233
Epoch 3/5
438/438 [==============================] - 106s 242ms/step - loss: 0.1382 - accuracy: 0.9448 - val_loss: 0.1657 - val_accuracy: 0.9283
Epoch 4/5
438/438 [==============================] - 97s 221ms/step - loss: 0.1064 - accuracy: 0.9552 - val_loss: 0.1794 - val_accuracy: 0.9297
Epoch 5/5
438/438 [==============================] - 101s 231ms/step - loss: 0.0870 - accuracy: 0.9619 - val_loss: 0.1750 - val_accuracy: 0.9283


# d) Performance analysis based on testing dataset (find precision, recall, F1-Score and accuracy) of the model and analysis the result

In [ ]:
from sklearn.metrics import classification_report

test_dataset = test_dataset.batch(32)
val_results = model.evaluate(val_dataset)

pred = model.predict(x_test)
pred_labels = tf.argmax(pred.logits, axis=1)
true_labels = y_test

classification_rep = classification_report(true_labels, pred_labels.numpy())
print(classification_rep)

94/94 [==============================] - 7s 71ms/step
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       430
           1       0.90      0.90      0.90       343
           2       0.95      0.97      0.96      1013
           3       0.91      0.82      0.87       254
           4       0.96      0.98      0.97       851
           5       0.84      0.75      0.79       109

    accuracy                           0.94      3000
   macro avg       0.92      0.90      0.91      3000
weighted avg       0.94      0.94      0.94      3000



Classification performance digunakan untuk mengevaluasi seberapa baik model klasifikasi dalam memprediksi kelas atau label dari data yang diberikan. Precision mengukur sejauh mana prediksi positif model akurat. Recall mengukur sejauh mana model mampu mendeteksi semua kasus positif. F1-score mengukur rata-rata harmonik antara presisi dan recall. Accuracy mengukur persentase prediksi yang benar dari keseluruhan prediksi.<br><br>
Berdasarkan hasil di atas dapat dilihat bahwa:<br>
- Model mampu memprediksi positif dengan benar sebesar 96%
- Model mampu memprediksi semua kasus positif sekitas 94%
- Keseimbangan antara precision dan recall sekitas 95%
- Model memiliki tingkat akurasi sekitas 94%

**Referensi**:<br>
Devlin, J., Chang, M.-W., Lee, K., Google, K. T., & Language, A. I. (n.d.). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. https://github.com/tensorflow/tensor2tensor <br>
Sains, H. F., & Teknologi, D. (2023). PEMODELAN TOPIK DALAM AL-QUR’AN MENGGUNAKAN LIBRARY BERTOPIC PADA MODEL BAHASA BERT. Jurnal SIMETRIS, 14(2). https://doi.org/10.24176/simet.v14i2.9900.<br>
https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/